# SIG_GIS / CMASIG Full Migration Guide (CMASIG.mdb → PostGIS → `gis_perimeters`)

This notebook documents the complete migration pipeline you are using in the SIGAM project:

- Exporting layers from `CMASIG.mdb` with `ogr2ogr`.
- Sanitising the raw SQL with `convrt.py`.
- Loading into the `sig_gis` PostGIS database.
- Importing SICMA demandes and building `sicma_demande_link`.
- Extending and synchronising `gis_perimeters` so that **all titles in `cmasig_titres`** are represented with the important metadata.

You can run the commands step by step from your shell using this notebook as a structured reference.

---

## 0. Prerequisites

- PostgreSQL + PostGIS installed.
- GDAL/OGR installed (`ogr2ogr` available in PATH).
- Working directory (repo root):

```bash
cd c:\Users\A\Desktop\sigam_vite\sigam
```

- Access database file: `CMASIG.mdb` (in the repo root or adjust the path).
- PostgreSQL connection parameters:
  - Database: `sig_gis`
  - User: `postgres`
  - Host: `localhost`
  - Password: your Postgres password

---

## 1. Create `sig_gis` database and enable PostGIS

From a terminal:

```bash
createdb -U postgres sig_gis

psql -d sig_gis -U postgres -p 5432 -c "CREATE EXTENSION IF NOT EXISTS postgis;"
psql -d sig_gis -U postgres -p 5433 -c "CREATE EXTENSION IF NOT EXISTS postgis_topology;"
```

---

## 2. Export CMASIG.mdb tables to raw SQL using `ogr2ogr`

We use `ogr2ogr` in **PGDump** mode to generate `.sql` files (one per CMASIG layer).

From the repo root:

```bash
cd c:\Users\A\Desktop\sigam_vite\sigam

# One‑time env (if not already set)
$env:GDAL_DATA = 'C:\gdal\bin\gdal-data'
$env:PROJ_LIB  = 'C:\gdal\bin\proj9\share'
$env:PATH = "C:\gdal\bin;C:\gdal\bin\gdal;C:\gdal\bin\gdal\apps;C:\gdal\bin\proj9\bin;$env:PATH"
# Wilayas
# One‑time env (if not already set)
$e
$env:PROJ_LIB  = 'C:nv:GDAL_DATA = 'C:\gdal\bin\gdal-data'\gdal\bin\proj9\share'
$env:PATH = "C:\gdal\bin;C:\gdal\bin\gdal;C:\gdal\bin\gdal\apps;C:\gdal\bin\proj9\bin;$env:PATH"

# Wilayas
& "C:\gdal\bin\gdal\apps\ogr2ogr.exe" -f "PGDump" "CMASIG_wilayas.sql" "CMASIG.mdb" Wilayas -nln cmasig_wilayas -lco GEOMETRY_NAME=geom -lco FID=objectid

# Communes
& "C:\gdal\bin\gdal\apps\ogr2ogr.exe" -f "PGDump" "CMASIG_communes.sql" "CMASIG.mdb" communes -nln cmasig_communes -lco GEOMETRY_NAME=geom -lco FID=objectid

# Villes
& "C:\gdal\bin\gdal\apps\ogr2ogr.exe" -f "PGDump" "CMASIG_villes.sql" "CMASIG.mdb" Villes -nln cmasig_villes -lco GEOMETRY_NAME=geom -lco FID=objectid

# Exclusion
& "C:\gdal\bin\gdal\apps\ogr2ogr.exe" -f "PGDump" "CMASIG_exclusion.sql" "CMASIG.mdb" Exclusion -nln cmasig_exclusion -lco GEOMETRY_NAME=geom -lco FID=objectid

# Modifications
& "C:\gdal\bin\gdal\apps\ogr2ogr.exe" -f "PGDump" "CMASIG_modifications.sql" "CMASIG.mdb" Modifications -nln cmasig_modifications -lco GEOMETRY_NAME=geom -lco FID=objectid

# Promotion
& "C:\gdal\bin\gdal\apps\ogr2ogr.exe" -f "PGDump" "CMASIG_promotion.sql" "CMASIG.mdb" Promotion -nln cmasig_promotion -lco GEOMETRY_NAME=geom -lco FID=objectid

# Titres
& "C:\gdal\bin\gdal\apps\ogr2ogr.exe" -f "PGDump" "CMASIG_titres.sql" "CMASIG.mdb" Titres -nln cmasig_titres -lco GEOMETRY_NAME=geom -lco FID=objectid

# Communes  double 
& "C:\gdal\bin\gdal\apps\ogr2ogr.exe" -f "PGDump" "CMASIG_communes.sql" "CMASIG.mdb" communes -nln cmasig_communes -lco GEOMETRY_NAME=geom -lco FID=objectid

# Villes  double 
& "C:\gdal\bin\gdal\apps\ogr2ogr.exe" -f "PGDump" "CMASIG_villes.sql" "CMASIG.mdb" Villes -nln cmasig_villes -lco GEOMETRY_NAME=geom -lco FID=objectid

# Exclusion    double 
& "C:\gdal\bin\gdal\apps\ogr2ogr.exe" -f "PGDump" "CMASIG_exclusion.sql" "CMASIG.mdb" Exclusion -nln cmasig_exclusion -lco GEOMETRY_NAME=geom -lco FID=objectid

# Modifications    double 
& "C:\gdal\bin\gdal\apps\ogr2ogr.exe" -f "PGDump" "CMASIG_modifications.sql" "CMASIG.mdb" Modifications -nln cmasig_modifications -lco GEOMETRY_NAME=geom -lco FID=objectid

# Promotion    double
& "C:\gdal\bin\gdal\apps\ogr2ogr.exe" -f "PGDump" "CMASIG_promotion.sql" "CMASIG.mdb" Promotion -nln cmasig_promotion -lco GEOMETRY_NAME=geom -lco FID=objectid

# Titres   double 
& "C:\gdal\bin\gdal\apps\ogr2ogr.exe" -f "PGDump" "CMASIG_titres.sql" "CMASIG.mdb" Titres -nln cmasig_titres -lco GEOMETRY_NAME=geom -lco FID=objectid

#Pays
& "C:\gdal\bin\gdal\apps\ogr2ogr.exe" -f "PGDump" "CMASIG_pays.sql" "CMASIG.mdb" Pays -nln cmasig_pays -lco GEOMETRY_NAME=geom -lco FID=objectid

```

This generates the raw SQL files:

- `CMASIG_wilayas.sql`
- `CMASIG_communes.sql`
- `CMASIG_villes.sql`
- `CMASIG_exclusion.sql`
- `CMASIG_modifications.sql`
- `CMASIG_promotion.sql`
- `CMASIG_titres.sql`

---

## 3. Sanitize raw SQL files to `_clean.sql` (UTF‑​8 safe)

`convrt.py` is a small utility that keeps:

- Geometry hex strings.
- ASCII attributes (ids, codes, labels).

and replaces any legacy control bytes / non-ASCII junk with spaces so that the result is safe UTF‑​8.

From the repo root:

```bash
cd c:\Users\A\Desktop\sigam_vite\sigam

python convrt.py CMASIG_wilayas.sql CMASIG_communes.sql CMASIG_villes.sql CMASIG_exclusion.sql CMASIG_modifications.sql CMASIG_promotion.sql CMASIG_titres.sql
```

This creates:

- `CMASIG_wilayas_clean.sql`
- `CMASIG_communes_clean.sql`
- `CMASIG_villes_clean.sql`
- `CMASIG_exclusion_clean.sql`
- `CMASIG_modifications_clean.sql`
- `CMASIG_promotion_clean.sql`
- `CMASIG_titres_clean.sql`

If you also have `CMASIG_titres_utf8.sql` and prefer that for titles, you can use it instead of `CMASIG_titres_clean.sql` in the next step.

---

## 4. Load sanitized CMASIG tables into `sig_gis`

```bash
cd c:\Users\A\Desktop\sigam_vite\sigam

psql -d sig_gis -U postgres -p 5433 -f CMASIG_wilayas_clean.sql
psql -d sig_gis -U postgres -p 5433 -f CMASIG_communes_clean.sql
psql -d sig_gis -U postgres -p 5433 -f CMASIG_villes_clean.sql
psql -d sig_gis -U postgres -p 5433 -f CMASIG_exclusion_clean.sql
psql -d sig_gis -U postgres -p 5433 -f CMASIG_modifications_clean.sql
psql -d sig_gis -U postgres -p 5433 -f CMASIG_promotion_clean.sql

:: For titres (choose one)
psql -d sig_gis -U postgres -p 5433 -f CMASIG_titres_clean.sql
:: or
:: 
psql -d sig_gis -U postgres -p 5433 -f CMASIG_titres_utf8.sql
```

You should now have PostGIS tables:

- `cmasig_wilayas`
- `cmasig_communes`
- `cmasig_villes`
- `cmasig_exclusion`
- `cmasig_modifications`
- `cmasig_promotion`
- `cmasig_titres`

---

## 5. Import SICMA demandes CSV and build `sicma_demande_link`

`sig_gis_sicma_link.sql` handles importing `df_demandeGeneral.csv` and building the link table.

```bash
cd c:\Users\A\Desktop\sigam_vite\sigam

psql -d sig_gis -U postgres -p 5433 -f sig_gis_sicma_link.sql
```

Inside, it:

- Creates `sicma_demande_general_raw` (all text columns).
- Truncates it.
- `\copy` from `BaseSicma_Urgence/df_demandeGeneral.csv` (separator `;`).
- Creates and fills `sicma_demande_link (id_demande, idtitre, id_proc)`.

Quick check:

```bash
psql -d sig_gis -U postgres -p 5433 -c "SELECT COUNT(*) FROM sicma_demande_link;"
```

---

## 6. Ensure `gis_perimeters` table and metadata columns exist


\set ON_ERROR_STOP 1

--
-- Synchronise all cmasig_titres into gis_perimeters
-- so that every title has a corresponding perimeter row
-- with the main metadata filled.
--
-- Run against the sig_gis database, e.g.:
--   psql -d sig_gis -f sig_gis_sync_titres_to_gis_perimeters.sql
--

BEGIN;

--
-- Name: gis_perimeters; Type: TABLE; Schema: public; Owner: -
--

CREATE TABLE public.gis_perimeters (
    id integer NOT NULL,
    sigam_proc_id integer,
    sigam_permis_id integer,
    source text NOT NULL,
    type_code text,
    status text,
    geom public.geometry(MultiPolygon,4326) NOT NULL,
    created_at timestamp with time zone DEFAULT now(),
    updated_at timestamp with time zone DEFAULT now(),
    permis_code text,
    permis_type_code text,
    permis_type_label text,
    permis_titulaire text,
    permis_area_ha double precision
);


--
-- Name: gis_perimeters_id_seq; Type: SEQUENCE; Schema: public; Owner: -
--

CREATE SEQUENCE public.gis_perimeters_id_seq
    AS integer
    START WITH 1
    INCREMENT BY 1
    NO MINVALUE
    NO MAXVALUE
    CACHE 1;


--
-- Name: gis_perimeters_id_seq; Type: SEQUENCE OWNED BY; Schema: public; Owner: -
--

ALTER SEQUENCE public.gis_perimeters_id_seq OWNED BY public.gis_perimeters.id;

-- Ensure id uses the sequence by default (important when table already exists)
ALTER TABLE ONLY public.gis_perimeters
  ALTER COLUMN id SET DEFAULT nextval('public.gis_perimeters_id_seq'::regclass);

-- 0) Ensure gis_perimeters has the metadata columns (idempotent)
ALTER TABLE gis_perimeters
  ADD COLUMN IF NOT EXISTS permis_code text,
  ADD COLUMN IF NOT EXISTS permis_type_code text,
  ADD COLUMN IF NOT EXISTS permis_type_label text,
  ADD COLUMN IF NOT EXISTS permis_titulaire text,
  ADD COLUMN IF NOT EXISTS permis_area_ha double precision;

-- Align the sequence value to current max(id) to avoid collisions
SELECT setval(
  'public.gis_perimeters_id_seq',
  COALESCE((SELECT max(id) FROM public.gis_perimeters), 0) + 1,
  false
);

-- 1) For perimeters already linked to a SIGAM procedure via sicma_demande_link,
--    fill sigam_permis_id and permit metadata from cmasig_titres when missing.

UPDATE gis_perimeters gp
SET
  sigam_permis_id    = COALESCE(gp.sigam_permis_id, t.idtitre),
  permis_code        = COALESCE(gp.permis_code, t.code::text),
  permis_type_code   = COALESCE(gp.permis_type_code, t.codetype),
  permis_type_label  = COALESCE(gp.permis_type_label, t.typetitre),
  permis_titulaire   = COALESCE(
                         gp.permis_titulaire,
                         trim(coalesce(t.tnom, '') || ' ' || coalesce(t.tprenom, ''))
                       ),
  permis_area_ha     = COALESCE(gp.permis_area_ha, t.sig_area)
FROM sicma_demande_link l
JOIN cmasig_titres t ON t.idtitre = l.idtitre
WHERE gp.sigam_proc_id = l.id_proc;

-- 2a) Insert perimeters for (procedure, titre) combinations that do not yet exist.
--     These are titles that have a link to a procedure (sicma_demande_link)
--     but no corresponding gis_perimeters row.

INSERT INTO gis_perimeters (
  sigam_proc_id,
  sigam_permis_id,
  source,
  type_code,
  status,
  geom,
  permis_code,
  permis_type_code,
  permis_type_label,
  permis_titulaire,
  permis_area_ha
)
SELECT
  l.id_proc                            AS sigam_proc_id,
  t.idtitre                            AS sigam_permis_id,
  'HISTORIC_TITRE'                     AS source,
  t.codetype                           AS type_code,
  NULL                                 AS status,
  ST_Multi(t.geom)                     AS geom,
  t.code::text                         AS permis_code,
  t.codetype                           AS permis_type_code,
  t.typetitre                          AS permis_type_label,
  trim(coalesce(t.tnom, '') || ' ' || coalesce(t.tprenom, '')) AS permis_titulaire,
  t.sig_area                           AS permis_area_ha
FROM sicma_demande_link l
JOIN cmasig_titres t
  ON t.idtitre = l.idtitre
LEFT JOIN gis_perimeters gp
  ON gp.sigam_proc_id   = l.id_proc
 AND (gp.sigam_permis_id = t.idtitre
      OR gp.permis_code  = t.code::text)
WHERE gp.id IS NULL;

-- 2b) Insert one perimeter per titre that still has no perimeter at all
--     (no existing gis_perimeters row for this idtitre / code).

INSERT INTO gis_perimeters (
  sigam_proc_id,
  sigam_permis_id,
  source,
  type_code,
  status,
  geom,
  permis_code,
  permis_type_code,
  permis_type_label,
  permis_titulaire,
  permis_area_ha
)
SELECT
  NULL                                 AS sigam_proc_id,
  t.idtitre                            AS sigam_permis_id,
  'CMASIG_TITRES'                      AS source,
  t.codetype                           AS type_code,
  NULL                                 AS status,
  ST_Multi(t.geom)                     AS geom,
  t.code::text                         AS permis_code,
  t.codetype                           AS permis_type_code,
  t.typetitre                          AS permis_type_label,
  trim(coalesce(t.tnom, '') || ' ' || coalesce(t.tprenom, '')) AS permis_titulaire,
  t.sig_area                           AS permis_area_ha
FROM cmasig_titres t
LEFT JOIN gis_perimeters gp
  ON gp.sigam_permis_id = t.idtitre
WHERE gp.id IS NULL
  AND t.geom IS NOT NULL
  AND NOT ST_IsEmpty(t.geom);

-- 2c) For titres that have EMPTY geometry in cmasig_titres but there is already
--     at least one historic perimeter with the same code, create a perimeter
--     by copying the geometry from an existing gis_perimeters row sharing
--     the same permis_code.

INSERT INTO gis_perimeters (
  sigam_proc_id,
  sigam_permis_id,
  source,
  type_code,
  status,
  geom,
  permis_code,
  permis_type_code,
  permis_type_label,
  permis_titulaire,
  permis_area_ha
)
WITH src AS (
  SELECT DISTINCT ON (t.idtitre)
    t.idtitre,
    t.code,
    t.codetype,
    t.typetitre,
    t.tnom,
    t.tprenom,
    t.sig_area,
    gp.sigam_proc_id      AS src_proc_id,
    gp.geom               AS src_geom,
    gp.permis_area_ha     AS src_area_ha
  FROM cmasig_titres t
  JOIN gis_perimeters gp
    ON gp.permis_code = t.code::text
  WHERE (t.geom IS NULL OR ST_IsEmpty(t.geom))
    AND NOT EXISTS (
      SELECT 1
      FROM gis_perimeters gp2
      WHERE gp2.sigam_permis_id = t.idtitre
    )
  ORDER BY t.idtitre, gp.id
)
SELECT
  src_proc_id                                     AS sigam_proc_id,
  idtitre                                         AS sigam_permis_id,
  'CMASIG_TITRES_FROM_HISTORIC'                   AS source,
  codetype                                        AS type_code,
  NULL                                            AS status,
  ST_Multi(src_geom)                              AS geom,
  code::text                                      AS permis_code,
  codetype                                        AS permis_type_code,
  typetitre                                       AS permis_type_label,
  trim(coalesce(tnom, '') || ' ' || coalesce(tprenom, '')) AS permis_titulaire,
  COALESCE(sig_area, src_area_ha)                 AS permis_area_ha
FROM src;

-- 3) Simple stats so you can see the result.

SELECT
  (SELECT COUNT(*) FROM cmasig_titres)                                  AS nb_titres,
  (SELECT COUNT(*) FROM gis_perimeters)                                  AS nb_perims_total,
  (SELECT COUNT(DISTINCT sigam_permis_id) FROM gis_perimeters
    WHERE sigam_permis_id IS NOT NULL)                                  AS nb_perims_with_titre;

COMMIT;


If `gis_perimeters` isn’t present yet, you can create it from the selected dump (`sig_gis_selected_tables.sql`) or your own schema.

To ensure it has the metadata columns used by your backend:

```bash
cd c:\Users\A\Desktop\sigam_vite\sigam

psql -d sig_gis -U postgres -p 5433 -f sig_gis_alter_gis_perimeters.sql
```

This script adds, if missing:

- `permis_code text`
- `permis_type_code text`
- `permis_type_label text`
- `permis_titulaire text`
- `permis_area_ha double precision`

---

## 7. Synchronise all `cmasig_titres` into `gis_perimeters`

This is the main consolidation step that ensures **every title** has a corresponding perimeter row with the important fields filled.

```bash
cd c:\Users\A\Desktop\sigam_vite\sigam

psql -d sig_gis -U postgres -p 5433 -f sig_gis_sync_titres_to_gis_perimeters.sql
```

### 7.1 What `sig_gis_sync_titres_to_gis_perimeters.sql` does

1. Adds metadata columns to `gis_perimeters` if they don’t already exist.
2. Updates existing `gis_perimeters` rows that are already linked to a procedure (`sigam_proc_id`) via `sicma_demande_link`, pulling metadata from `cmasig_titres`:
   - `sigam_permis_id`
   - `permis_code`
   - `permis_type_code`
   - `permis_type_label`
   - `permis_titulaire`
   - `permis_area_ha`
3. Inserts rows for all `(id_proc, idtitre)` pairs present in `sicma_demande_link` that don’t yet have a matching perimeter:
   - `source = 'HISTORIC_TITRE'`
   - `geom = ST_Multi(cmasig_titres.geom)`
4. Inserts one perimeter per titre that **has valid, non-empty geometry** but no perimeter at all (no `gis_perimeters` row with that `sigam_permis_id`):
   - `source = 'CMASIG_TITRES'`
   - `sigam_proc_id = NULL`
   - `geom = ST_Multi(t.geom)`
5. For titres with **empty geometry** in `cmasig_titres` but existing historic perimeters (same `code`), it copies geometry from those historic perimeters and inserts a new perimeter:
   - `source = 'CMASIG_TITRES_FROM_HISTORIC'`
   - `geom` copied from an existing `gis_perimeters` row where `permis_code = t.code::text`
6. Prints summary counts at the end:
   - `nb_titres` (rows in `cmasig_titres`)
   - `nb_perims_total` (rows in `gis_perimeters`)
   - `nb_perims_with_titre` (distinct `sigam_permis_id` in `gis_perimeters`)

---

## 8. Verification & Diagnostics

### 8.1 Global counts

```bash
psql -d sig_gis -U postgres -p 5433
```

Then inside `psql`:

```sql
SELECT
  (SELECT COUNT(*) FROM cmasig_titres) AS nb_titres,
  (SELECT COUNT(*) FROM gis_perimeters) AS nb_perims_total,
  (SELECT COUNT(DISTINCT sigam_permis_id)
   FROM gis_perimeters
   WHERE sigam_permis_id IS NOT NULL) AS nb_perims_with_titre;
```

### 8.2 List any titles that still have no perimeter

```sql
SELECT
  t.idtitre,
  t.code,
  t.typetitre,
  t.tnom,
  t.tprenom,
  t.geom IS NULL     AS geom_is_null,
  ST_IsEmpty(t.geom) AS geom_is_empty,
  ST_IsValid(t.geom) AS geom_is_valid
FROM cmasig_titres t
LEFT JOIN gis_perimeters gp
  ON gp.sigam_permis_id = t.idtitre
WHERE gp.id IS NULL;
```

If any titres remain:

- If `geom_is_null` or `geom_is_empty` **and** there is no historic perimeter with the same code, you must fix `geom` in `cmasig_titres` or decide a manual polygon.
- Otherwise, you can inspect and manually insert/update in `gis_perimeters`.

### 8.3 Inspect a specific titre (example: `idtitre = 8043`)

```sql
SELECT * FROM cmasig_titres WHERE idtitre = 8043;

SELECT *
FROM gis_perimeters
WHERE sigam_permis_id = 8043
   OR permis_code = '6463';
```

If there is still no geometry for this titre and no historic perimeter to copy, you must:

- Either correct `geom` in `cmasig_titres` (if you know the right polygon), then re-run the sync script.
- Or insert a manual perimeter row in `gis_perimeters` using a polygon you decide.

---

With this notebook, you have a single, organised place that captures the entire migration process from `CMASIG.mdb` → PostGIS tables → `gis_perimeters`, including all key SQL and `ogr2ogr` commands used in your SIGAM project.

### flakher kaml dir had le script bach arcgis pro ye9ra correctlly les geometry ta3 postgis

\set ON_ERROR_STOP 1

BEGIN;

-- Helper to add a primary key if missing
DO $$
BEGIN
  IF NOT EXISTS (
    SELECT 1 FROM pg_constraint
    WHERE conrelid = 'public.cmasig_titres'::regclass
      AND contype  = 'p'
  ) THEN
    ALTER TABLE public.cmasig_titres ADD PRIMARY KEY (idtitre);
  END IF;
END$$;

DO $$
BEGIN
  IF NOT EXISTS (
    SELECT 1 FROM pg_constraint
    WHERE conrelid = 'public.cmasig_promotion'::regclass
      AND contype  = 'p'
  ) THEN
    ALTER TABLE public.cmasig_promotion ADD PRIMARY KEY (objectid);
  END IF;
END$$;

DO $$
BEGIN
  IF NOT EXISTS (
    SELECT 1 FROM pg_constraint
    WHERE conrelid = 'public.cmasig_exclusion'::regclass
      AND contype  = 'p'
  ) THEN
    ALTER TABLE public.cmasig_exclusion ADD PRIMARY KEY (objectid);
  END IF;
END$$;

DO $$
BEGIN
  IF NOT EXISTS (
    SELECT 1 FROM pg_constraint
    WHERE conrelid = 'public.cmasig_modifications'::regclass
      AND contype  = 'p'
  ) THEN
    ALTER TABLE public.cmasig_modifications ADD PRIMARY KEY (objectid);
  END IF;
END$$;

DO $$
BEGIN
  IF NOT EXISTS (
    SELECT 1 FROM pg_constraint
    WHERE conrelid = 'public.cmasig_communes'::regclass
      AND contype  = 'p'
  ) THEN
    ALTER TABLE public.cmasig_communes ADD PRIMARY KEY (objectid);
  END IF;
END$$;

DO $$
BEGIN
  IF NOT EXISTS (
    SELECT 1 FROM pg_constraint
    WHERE conrelid = 'public.cmasig_wilayas'::regclass
      AND contype  = 'p'
  ) THEN
    ALTER TABLE public.cmasig_wilayas ADD PRIMARY KEY (objectid);
  END IF;
END$$;

DO $$
BEGIN
  IF NOT EXISTS (
    SELECT 1 FROM pg_constraint
    WHERE conrelid = 'public.cmasig_villes'::regclass
      AND contype  = 'p'
  ) THEN
    ALTER TABLE public.cmasig_villes ADD PRIMARY KEY (objectid);
  END IF;
END$$;

-- Repair geometries and enforce types/SRID

UPDATE public.cmasig_titres
SET geom = ST_SetSRID(
              ST_Multi(
                ST_CollectionExtract(
                  ST_MakeValid(ST_Force2D(geom)),
                  3
                )
              ),
              4326
            )
WHERE geom IS NOT NULL AND NOT ST_IsValid(geom);

ALTER TABLE public.cmasig_titres
  ALTER COLUMN geom TYPE geometry(MultiPolygon,4326)
    USING ST_SetSRID(
            ST_Multi(
              ST_CollectionExtract(
                ST_MakeValid(ST_Force2D(geom)),
                3
              )
            ), 4326);

UPDATE public.cmasig_promotion
SET geom = ST_SetSRID(
              ST_Multi(
                ST_CollectionExtract(
                  ST_MakeValid(ST_Force2D(geom)),
                  3
                )
              ),
              4326
            )
WHERE geom IS NOT NULL AND NOT ST_IsValid(geom);

ALTER TABLE public.cmasig_promotion
  ALTER COLUMN geom TYPE geometry(MultiPolygon,4326)
    USING ST_SetSRID(
            ST_Multi(
              ST_CollectionExtract(
                ST_MakeValid(ST_Force2D(geom)),
                3
              )
            ), 4326);

UPDATE public.cmasig_exclusion
SET geom = ST_SetSRID(
              ST_Multi(
                ST_CollectionExtract(
                  ST_MakeValid(ST_Force2D(geom)),
                  3
                )
              ),
              4326
            )
WHERE geom IS NOT NULL AND NOT ST_IsValid(geom);

ALTER TABLE public.cmasig_exclusion
  ALTER COLUMN geom TYPE geometry(MultiPolygon,4326)
    USING ST_SetSRID(
            ST_Multi(
              ST_CollectionExtract(
                ST_MakeValid(ST_Force2D(geom)),
                3
              )
            ), 4326);

UPDATE public.cmasig_modifications
SET geom = ST_SetSRID(
              ST_Multi(
                ST_CollectionExtract(
                  ST_MakeValid(ST_Force2D(geom)),
                  3
                )
              ),
              4326
            )
WHERE geom IS NOT NULL AND NOT ST_IsValid(geom);

ALTER TABLE public.cmasig_modifications
  ALTER COLUMN geom TYPE geometry(MultiPolygon,4326)
    USING ST_SetSRID(
            ST_Multi(
              ST_CollectionExtract(
                ST_MakeValid(ST_Force2D(geom)),
                3
              )
            ), 4326);

UPDATE public.cmasig_communes
SET geom = ST_SetSRID(
              ST_Multi(
                ST_CollectionExtract(
                  ST_MakeValid(ST_Force2D(geom)),
                  3
                )
              ),
              4326
            )
WHERE geom IS NOT NULL AND NOT ST_IsValid(geom);

ALTER TABLE public.cmasig_communes
  ALTER COLUMN geom TYPE geometry(MultiPolygon,4326)
    USING ST_SetSRID(
            ST_Multi(
              ST_CollectionExtract(
                ST_MakeValid(ST_Force2D(geom)),
                3
              )
            ), 4326);

UPDATE public.cmasig_wilayas
SET geom = ST_SetSRID(
              ST_Multi(
                ST_CollectionExtract(
                  ST_MakeValid(ST_Force2D(geom)),
                  3
                )
              ),
              4326
            )
WHERE geom IS NOT NULL AND NOT ST_IsValid(geom);

ALTER TABLE public.cmasig_wilayas
  ALTER COLUMN geom TYPE geometry(MultiPolygon,4326)
    USING ST_SetSRID(
            ST_Multi(
              ST_CollectionExtract(
                ST_MakeValid(ST_Force2D(geom)),
                3
              )
            ), 4326);

UPDATE public.cmasig_villes
SET geom = ST_SetSRID(
              ST_MakeValid(ST_Force2D(geom)),
              4326
            )
WHERE geom IS NOT NULL AND NOT ST_IsValid(geom);

ALTER TABLE public.cmasig_villes
  ALTER COLUMN geom TYPE geometry(Point,4326)
    USING ST_SetSRID(
            ST_MakeValid(ST_Force2D(geom)),
            4326);

-- gis_perimeters geometry and sequence alignment
UPDATE public.gis_perimeters
SET geom = ST_SetSRID(
              ST_Multi(
                ST_CollectionExtract(
                  ST_MakeValid(ST_Force2D(geom)),
                  3
                )
              ),
              4326
            )
WHERE geom IS NOT NULL AND NOT ST_IsValid(geom);

ALTER TABLE public.gis_perimeters
  ALTER COLUMN geom TYPE geometry(MultiPolygon,4326)
    USING ST_SetSRID(
            ST_Multi(
              ST_CollectionExtract(
                ST_MakeValid(ST_Force2D(geom)),
                3
              )
            ), 4326);

ALTER TABLE ONLY public.gis_perimeters
  ALTER COLUMN id SET DEFAULT nextval('public.gis_perimeters_id_seq'::regclass);

SELECT setval(
  'public.gis_perimeters_id_seq',
  COALESCE((SELECT max(id) FROM public.gis_perimeters), 0) + 1,
  false
);

-- Indexes for performance / ArcGIS
CREATE INDEX IF NOT EXISTS idx_cmasig_titres_geom       ON public.cmasig_titres       USING GIST (geom);
CREATE INDEX IF NOT EXISTS idx_cmasig_promotion_geom    ON public.cmasig_promotion    USING GIST (geom);
CREATE INDEX IF NOT EXISTS idx_cmasig_exclusion_geom    ON public.cmasig_exclusion    USING GIST (geom);
CREATE INDEX IF NOT EXISTS idx_cmasig_modifications_geom ON public.cmasig_modifications USING GIST (geom);
CREATE INDEX IF NOT EXISTS idx_cmasig_communes_geom     ON public.cmasig_communes     USING GIST (geom);
CREATE INDEX IF NOT EXISTS idx_cmasig_wilayas_geom      ON public.cmasig_wilayas      USING GIST (geom);
CREATE INDEX IF NOT EXISTS idx_cmasig_villes_geom       ON public.cmasig_villes       USING GIST (geom);
CREATE INDEX IF NOT EXISTS idx_gis_perimeters_geom      ON public.gis_perimeters      USING GIST (geom);

COMMIT;

###############################################################
CREATE TABLE corbielle_promotion (
    objectid       INTEGER        NOT NULL,
    geom           geometry(MultiPolygon,4326),
    idzone         NUMERIC,
    nom            VARCHAR(250),
    sig_area       DOUBLE PRECISION,
    shape_length   DOUBLE PRECISION,
    shape_area     DOUBLE PRECISION,
    deleted_by     VARCHAR(150),
    commentaire    TEXT,
    deleted_at     TIMESTAMP DEFAULT NOW()
);

